# Generator

### reference<br>
-------------------------------------------------
https://docs.python.org/3/library/itertools.html#module-itertools

### generator를 배우기 앞서
--------------------------------------------------
1. procedural<br>
2. object-oriented<br>
3. functional<br>
위 세 가지의 프로그래밍 방법이 있다.

functional programming은 문제를 여러가지의 함수로 분해한다.<br>
그리고 위 함수들은 어떠한 output에 영향을 주는 internal state없이 input이 주어지면<br>
같은 output을 생성하도록 구현된다.<br>

python은 위 세 가지의 방법 모두를 지원하며 꼭 위 세가지 중 하나를 선택할 필요는 없다.<br>
functional programming을 위해서는 I/O도, assignment도 하지 않아야하는 극단적인<br>
선택을 해야하는데 파이썬은 그런 방법을 선택하지 않고<br>
__functional-approaching-interface__를 제공한다.<br>

Object-oriented programming이 작은 capule 안에서 method를 호출하며<br>
internal state를 바꿔가 프로그램이 진행된다면

Functional programming은 internal-state의 변화를 최소화하며<br>
오직 함수 사이의 data flow 만으로 문제를 해결하는 방법이다.

Python에서는 위 두가지 방법 모두를 사용한다.

# iterator

- 먼저 functional programming의 foundation인 iterator를 살펴보자

1. iterator는 stream of data를 나타내는 object이다.<br>
2. iterator는 반드시 \_\_next__ magic method가 있어야하며 arguments는 없고<br>
   항상 다음 elements를 return한다.<br>
   만약 다음 elem이 없으면 StopIteration을 raise한다.<br>
3. 꼭 finite하게 구현할 필요는 없으며 infinite하게 구현하는 것이 훨씬 낫다.

- built-int function인 \_\_iter__는 arbitrary object을 args로 받는다.<br>
  만약 object이 iteration을 지원하지 않는다면 _TypeError_를 raise한다.
- iterator가 존재하는 object을 __iterable__이라고 한다.

In [2]:
L = [1, 2, 3]
it = iter(L)
print(next(it))
print(next(it))
print(next(it))
print(next(it))

1
2
3


StopIteration: 

python에서는 여러 경우의 iterator를 요구하는데<br>
대표적인 예가 for문이다.
```python
    for X in Y
```
에서 Y는 iterator를 꼭 가져야한다.<br>
아래 두 코드는 동일하다.<br>
```python
    for i in iter(obj):
        print(i)

    for i in obj:
        print(i)
```

max(), min() built-in function을 사용할 수 있으며
_in_, _not in_ 도 iterator가 정의되어야한다.

- python에서 지원하는 대부분의 sequential은 iterator를 지원한다.
- list, tuple, dictionary, string도 내부적으로 iteration을 생성한다.
- 특별히 dictionary 에서는 _keys()_, _values()_ 라는 method로<br>
  다른 iter를 생성할 수 있다.

- iter가 output을 생성하는 방법에는 보통 두 가지가 있다.
1. 모든 elements에 대하여 각각 operate<br>
2. 특정 조건을 만족하는 subset에 대하여 operate<br>

- 짧게 "listcomp", "genexps"이라고 하는
list comprehension, generator expressions 는 functional programming language인<br>
Haskell에서 빌려온 표현이다.

In [5]:
line_list = ['     line1  \n', '    line2  \n', 'line3    \n', '']

# generator expression
stripped_iter = (line.strip() for line in line_list)
# list comprehension
stripped_list = [line.strip() for line in line_list]

print(stripped_iter)
print(stripped_list)

<generator object <genexpr> at 0x7ff7494b0f20>
['line1', 'line2', 'line3', '']


특정 조건에 맞는 element만 뽑고 싶다면 아래와 같이 작성할 수도 있다.<br>

In [6]:
stripped_condition_list = [
    line.strip() for line in line_list if line != ''
]
print(stripped_condition_list)

['line1', 'line2', 'line3']


```python
    (expression for exp1 in seq1
                if condition1
                for exp2 in seq2
                if condition2
                for expN in seqN
                if conditionN )
```
위 genexps는 아래의 코드 진행 순서와 일치한다.<br>
```python
    for exp1 in seq1:
        if not condition1:
            continue
        for exp2 in seq2:
            if not condition2:
                continue
            ...
            for expN in seqN:
                if not conditionN:
                    continue
```

- python이 모호하게 해석하지 않게 만약 generator가 tuple을 생성한다면<br>
  () 로 감싸줘야한다.
```python
    [(x, y) for x in seq1 for y in seq2]
```

Generator의 기본 아이디어는 다음과 같다.<br>
기존의 C, python이 함수를 호출하는 방식은 각 함수만의 private namespace를 생성하고<br>
내부 계산을 수행한 뒤 결과를 return하고 생성한 namespace를 날려버린다.<br>
그리고 다음에 다시 함수가 호출되면 새로 private namespace를 생성하고 같은 작업을 반복한다.<br>

그러나 만약 함수가 그 private namespace를 날리지 않고 잠시 멈췄다가 다시 실행할 수 있다면..?<br>
이것이 generator 함수의 기본 아이디어이다.<br>

In [8]:
def generate_ints(N):
    for i in range(N):
        yield i

- __yeild__ keyword가 있는 모든 함수는 generator 함수이다.

보통 아래와 같이 사용한다.

In [9]:
gen = generate_ints(3)
print(next(gen))
print(next(gen))
print(next(gen))
print(next(gen))

0
1
2


StopIteration: 

# Passing values into generator

value는 send(value) method를 call함으로써 generator안에 전달될 수 있다.<br>
send(value)는 generator code를 실행하고 value를 generator 안에서 return한다.

아래 예제를 살펴보자

In [12]:
def counter(N):
    i = 0
    while i <= N:
        val = (yield i)
        if val is not None:
            i = val
        else:
            i += 1

it = counter(10)
print(next(it))
print(next(it))
print(next(it))
it.send(5)
print(next(it))
print(next(it))
print(next(it))

0
1
2
6
7
8


위 generator code 안에서 yield는 일반적인 \_\_next__ method는
None을 return한다.






































